In [ ]:
# Code to mount google drive and change working directory
from google.colab import drive
drive.mount('/content/drive')
%ls
%cd drive/
%ls
%cd My\ Drive
%ls
%cd Colab\ Notebooks
%ls



Mounted at /content/drive
drive/  sample_data/
/content/drive
MyDrive/
/content/drive/My Drive
'16012023 SA8 056.pdf'                         'Investor Information Form.gdoc'
'AlexOJ consent form (1).pdf'                  'Isreal Osaretin CV 2.docx'
'AlexOJ consent form (2).pdf'                  'Mandate SW.docx'
'AlexOJ consent form (3).pdf'                  'New Start Payroll Form (PAYE) (002).docx'
'AlexOJ consent form.pdf'                      'SMART INDUCTION PACK JAN 2021 (WFH) (1).pdf'
'Colab Notebooks'/                             'Starter-checklist_English_2021_2022 (1).pdf'
 CV.docx                                        Starter-checklist_English_2021_2022.gdoc
'Fee Assessment Questionnaire Version 2.docx'   Starter-checklist_English_2021_2022.pdf
'Fee Assessment Questionnaire Version 2.gdoc'  'Untitled document.gdoc'
'Fee Status Questionaire.gdoc'                 'Untitled form.gform'
'Fee Status Questionaire.pdf'                  'unturned audio'
'How to get started with Dr

In [ ]:
### Google News Headline Scrapper

In [ ]:
!pip install feedparser
!pip install azure-ai-textanalytics


import pandas as pd
from datetime import datetime, timedelta
import feedparser

# Define your start and end dates for filtering
start_date = datetime(2024, 3, 28)
end_date = datetime(2024, 4, 4)

rss_url = "https://news.google.com/rss/search?q=%22S%26P+500%22+OR+%22SPX%22+OR+%22$SPY%22&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(rss_url)

articles_by_date = {}

for entry in feed.entries:
    published_date = datetime(*entry.published_parsed[:6])

    # Filter articles by the specified start and end dates
    if start_date <= published_date <= end_date:
        date_str = published_date.strftime('%Y-%m-%d')
        headline = entry.title

        if date_str in articles_by_date:
            articles_by_date[date_str].append(headline)
        else:
            articles_by_date[date_str] = [headline]

data_for_df = []
for date, headlines in articles_by_date.items():
    row = {'date': date}
    for i, headline in enumerate(headlines):
        row[f'headline_{i+1}'] = headline
    data_for_df.append(row)

df_articles = pd.DataFrame(data_for_df)
df_articles = df_articles.fillna('')

print(df_articles.head())

output_file_name = 'google_news_headlines_by_date.csv'
df_articles.to_csv(output_file_name, index=False)

print(f"Filtered and saved articles to {output_file_name}.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=a64c9e7adef149210be24074360ce1738e12c19594e79ca6f07e03059f87dd88
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.5 MB/s eta 0:00:00
         date                                         headline_1  \
0  2024-04-03  S&P 500 Snaps Two-Day Losing Streak - WSJ - Th...   
1  2024-04-02  S&P 500 Gains and Losses Today: Health Care St...   
2  2024-04-01  S&P 500 Gains and Losses Today: Shares of Memo...   
3  2024-03-31  The Magnificent 7 are no longer the only stock...   
4  

In [ ]:
!pip install azure-ai-textanalytics

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import pandas as pd

# Set up the Azure Text Analytics client
def authenticate_client():
    endpoint = "https://sentimentanalysishp.cognitiveservices.azure.com/"  # Replace with your endpoint
    key = "dedffb82877445dd93d95092e65b54bc"  # Replace with your key
    return TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Analyze sentiment of documents
def analyze_sentiment(documents):
    client = authenticate_client()
    response = client.analyze_sentiment(documents=documents, language="en")
    return response

# Adjust the CSV path to match your Google Drive structure
csv_path = '/content/drive/My Drive/Colab Notebooks/google_news_headlines_by_date.csv'

# Read the CSV and process it for sentiment analysis
df = pd.read_csv(csv_path)

# Assuming you want to analyze the sentiment of the first headline column as an example
documents = df['headline_1'].dropna().tolist()  # Modify as needed for other columns

sentiment_responses = analyze_sentiment(documents)

for response in sentiment_responses:
    if not response.is_error:
        print(f"Document: {response.sentences[0].text}")
        print(f"Sentiment: {response.sentiment}, Scores: {response.confidence_scores}")
    else:
        print(f"Error: {response.error}")


Document: S&P 500 Snaps Two-Day Losing Streak - WSJ - The Wall Street Journal
Sentiment: neutral, Scores: {'positive': 0.02, 'neutral': 0.77, 'negative': 0.21}
Document: S&P 500 Gains and Losses Today: Health Care Stocks Move Lower - Investopedia
Sentiment: neutral, Scores: {'positive': 0.02, 'neutral': 0.85, 'negative': 0.13}
Document: S&P 500 Gains and Losses Today: Shares of Memory Chipmakers Push Higher - Investopedia
Sentiment: neutral, Scores: {'positive': 0.04, 'neutral': 0.9, 'negative': 0.07}
Document: The Magnificent 7 are no longer the only stocks driving S&P 500 to record highs - MarketWatch
Sentiment: neutral, Scores: {'positive': 0.08, 'neutral': 0.83, 'negative': 0.1}
Document: The S&P 500 just turned in its best first quarter since 2019 - CNN
Sentiment: neutral, Scores: {'positive': 0.39, 'neutral': 0.59, 'negative': 0.02}
Document: Beware Nvidia and the S&P 500 ‘index waltz,’ market-beating fund manager says - MarketWatch
Sentiment: neutral, Scores: {'positive': 0.07, 

In [ ]:
# import pandas as pd

# !pip install flair

# import flair
# import scipy.linalg
# print(scipy.linalg.triu)
# from textblob import TextBlob
# import os
# import datetime
# import numpy as np

# import nltk
# nltk.download('vader_lexicon')
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

# print("Sentiment analysis imports.")


In [ ]:
# import pandas as pd
# import flair

# flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

# negative_keywords = ['plunges', 'falls', 'down', 'losses', 'worst', 'declines', 'drops', 'suffers']

# def simple_sentiment_heuristic(headline):
#     """
#     Simple heuristic to detect negative sentiment based on specific keywords.
#     """
#     for keyword in negative_keywords:
#         if keyword in headline.lower():
#             return "NEGATIVE", 1.0
#     return None, None

# def get_sentiment_label(sentence):
#     """
#     Determine sentiment label based on the Flair model prediction.
#     """
#     flair_sentiment.predict(sentence)
#     sentiment_score = sentence.labels[0].score
#     if sentence.labels[0].value == 'NEGATIVE':
#         sentiment_score = 1 - sentiment_score
#     if sentiment_score > 0.55:
#         return "POSITIVE", sentiment_score
#     elif sentiment_score < 0.45:
#         return "NEGATIVE", sentiment_score
#     else:
#         return "NEUTRAL", sentiment_score

# def get_sentiment_report(input_filename, output_filename):
#     data_df = pd.read_csv(input_filename)
#     results = []

#     for index, row in data_df.iterrows():
#         date = row['date']
#         daily_sentiments = []
#         sentiment_confidences = []

#         for headline in row[1:].dropna():
#             heuristic_label, heuristic_confidence = simple_sentiment_heuristic(headline)
#             if heuristic_label:
#                 sentiment_label = heuristic_label
#                 sentiment_confidence = heuristic_confidence
#             else:
#                 sentence = flair.data.Sentence(headline)
#                 sentiment_label, sentiment_confidence = get_sentiment_label(sentence)

#             daily_sentiments.append(sentiment_label)
#             sentiment_confidences.append(sentiment_confidence)

#         overall_sentiment = "NEUTRAL"
#         if daily_sentiments:
#             positive_count = daily_sentiments.count("POSITIVE")
#             negative_count = daily_sentiments.count("NEGATIVE")
#             if positive_count > negative_count:
#                 overall_sentiment = "POSITIVE"
#             elif negative_count > positive_count:
#                 overall_sentiment = "NEGATIVE"

#         overall_confidence = sum(sentiment_confidences) / len(sentiment_confidences) if sentiment_confidences else 0

#         results.append([date, overall_sentiment, overall_confidence] + daily_sentiments)

#     max_headlines = max(len(r) - 3 for r in results)
#     column_names = ['date', 'overall_sentiment', 'confidence'] + [f'headline_{i}_sentiment' for i in range(1, max_headlines + 1)]
#     results_df = pd.DataFrame(results, columns=column_names)

#     results_df.to_csv(output_filename, index=False)
#     print(f"Sentiment analysis results saved to {output_filename}")

# if __name__ == "__main__":
#     input_filename = 'test_Sentiment_Data.csv'
#     output_filename = 'sentiment_analysis_results_2.csv'
#     get_sentiment_report(input_filename, output_filename)
